In [1]:
import torch

In [ ]:
# Создаем входной массив из двух изображений RGB 3*3
input_images = torch.tensor(
      [[[[0,  1,  2],
         [3,  4,  5],
         [6,  7,  8]],

        [[9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])


def get_padding2d(input_images):
    #padded_images = # добавить нулей с четырех сторон каждого изображения
    ## Top button
    t=list(input_images.size())
    t[2]=1
    newrow=torch.zeros(t)
    newrow=newrow.type(torch.long)
    input_images=input_images.type(torch.long)
    
    input_images=torch.cat((newrow, input_images), 2)
    input_images=torch.cat((input_images,newrow), 2)
    input_images


    ## Sides
    t=list(input_images.size())
    t[-1]=1
    newrow=torch.zeros(t)
    newrow=newrow.type(torch.long)
    input_images=input_images.type(torch.long)
    
    input_images=torch.cat((newrow, input_images), 3)
    input_images=torch.cat((input_images, newrow), 3)
    padded_images=input_images
    
    padded_images=padded_images.type(torch.float)
    return padded_images

In [ ]:
get_padding2d(input_images)

In [ ]:
torch.sum(tensor,dim)

In [ ]:
a=torch.randint(10, (2, 2))
b=torch.randint(10, (2, 2))

print(a,"\n\n",b,"\n\n", a+b, torch.sum(a+b))

## My Conv2d

In [2]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


In [ ]:
# Default tensor

In [37]:
kernel = torch.tensor(
                  [[[[0., 1, 0],
                     [1,  2, 1],
                     [0,  1, 0]],

                    [[1, 2, 1],
                     [0, 3, 3],
                     [0, 1, 10]],

                    [[10, 11, 12],
                     [13, 14, 15],
                     [16, 17, 18]]]])

kernel_w=kernel.shape[-1]
kernel_h=kernel.shape[-2]
print(kernel_w,kernel_h)

3 3


In [24]:
kernel.shape

torch.Size([1, 3, 3, 3])

In [39]:
torch.squeeze(kernel,dim=0).shape

torch.Size([3, 3, 3])

In [ ]:
# My tensor

In [30]:
my_kernel = torch.randint(10, (4, 3,3,3))
my_kernel.shape

torch.Size([4, 3, 3, 3])

In [23]:
kernel.shape

torch.Size([1, 3, 3, 3])

In [ ]:
for i in kernel:
    print(i)

In [40]:
# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):

    def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
        batch_size, channels_count, input_height, input_width = input_matrix_shape
        output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
        output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    
        return batch_size, out_channels, output_height, output_width

    def out_image_shape(self, input_height, input_width, kernel_size, stride, padding):
        output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
        output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1
        return output_height, output_width

    def create_empty_out_img(self, input_tensor):
        self.input_width = input_tensor.shape[-1]
        self.input_height = input_tensor.shape[-2]
        # in_chanels = input_tensor.shape[-3]
        
        padding = 0
        self.output_height, self.output_width = self.out_image_shape(self.input_height, self.input_width, self.kernel_size, self.stride, padding)

        # Cheking batch size 
        if len(input_tensor.shape) == 3: 
            batch_size = 1
        if len(input_tensor.shape) == 4:
            batch_size = input_tensor.shape[-4]
            
        out_tesor = torch.zeros([ batch_size, self.out_channels, self.output_height, self.output_width])
        return out_tesor

    def conv_one_sample(self, one_sample, filter, idx_batch, idx_filter):
        h_start = 0
        for _ in range(self.output_height): #h_centr
            w_start = 0
            for _ in range(self.output_width): # w_centr
                # print(f"self.kernel_size {self.kernel_size}")
                window_sliced = one_sample[:,   h_start:h_start+self.kernel_size,   w_start: w_start+self.kernel_size]
                # print(f"window_sliced {window_sliced}")
                # print(f"kernel[idx_filter] {self.kernel[idx_filter]}")
                # print(f"wind+kernel { window_sliced + self.kernel[idx_filter]}")
                self.out_tensor[ idx_batch, idx_filter, h_start, w_start] = torch.sum( window_sliced + self.kernel[idx_filter])
                # print(f"self.out_tensor {self.out_tensor}")
                w_start+=self.stride
                # print(f"\n\n")
            h_start+=self.stride
            
        
        
    def __call__(self, input_tensor):
        # output_tensor = # Напишите в этом месте вычисление свертки с использованием циклов.

        # create out_tensor
        self.out_tensor=self.create_empty_out_img(input_tensor)
        print(f"self.out_tensor {self.out_tensor.shape}")

        
        # Check tensor batch or not and unsqueeze if not
        if len(input_tensor.shape)==4:
            is_batch = True
        elif len(input_tensor.shape)==3:
            is_batch = False
            input_tensor = input_tensor.unsqueeze(0)

        # # Check tensor batch or not and unsqueeze if not
        # if len(input_tensor.shape)==4:
        #     is_batch = True
        # elif len(input_tensor.shape)==3:
        #     is_batch = False
        #     input_tensor = input_tensor.unsqueeze(0)

        # main loops
        for idx_batch, one_sample in enumerate(input_tensor): # itteration through batch 
            for idx_filter, filter in enumerate(self.kernel):   # itteration through kernel filters 
                self.conv_one_sample(one_sample, filter, idx_batch, idx_filter)

        # squeeze if not batch
        if is_batch:
            self.out_tensor = torch.squeeze(self.out_tensor,dim=0)
        
        return self.out_tensor

    

In [42]:
img = torch.randint(10, (2, 3, 3, 4)).to(torch.float32)
img

tensor([[[[1., 3., 4., 5.],
          [6., 1., 4., 3.],
          [6., 0., 4., 2.]],

         [[2., 6., 5., 6.],
          [8., 4., 4., 1.],
          [1., 7., 3., 2.]],

         [[5., 3., 1., 7.],
          [8., 6., 6., 6.],
          [6., 3., 9., 7.]]],


        [[[8., 5., 6., 7.],
          [1., 9., 2., 7.],
          [3., 5., 4., 6.]],

         [[9., 4., 7., 0.],
          [5., 5., 5., 8.],
          [6., 5., 4., 6.]],

         [[8., 4., 7., 5.],
          [0., 4., 4., 9.],
          [9., 5., 4., 8.]]]])

In [8]:
len(img.shape)

4

In [9]:
img.shape

torch.Size([8, 3, 4, 5])

## Get empty tensor

In [10]:
kernel

tensor([[[[ 0.,  1.,  0.],
          [ 1.,  2.,  1.],
          [ 0.,  1.,  0.]],

         [[ 1.,  2.,  1.],
          [ 0.,  3.,  3.],
          [ 0.,  1., 10.]],

         [[10., 11., 12.],
          [13., 14., 15.],
          [16., 17., 18.]]]])

In [35]:
in_channels = 3
out_channels = 4
kernel_size = 3
stride = 1


my_conv = Conv2dLoop(in_channels, out_channels, kernel_size, stride)
# my_conv.set_kernel(kernel)
my_conv.set_kernel(my_kernel)

# my_conv.create_empty_out_img(img)

# img = torch.randint(10, ( 3, 7, 10)).to(torch.float32)
# my_conv(img)

In [ ]:
img

In [ ]:
my_conv.kernel[0]

## Call my conv3d

In [36]:
my_conv(img)

self.out_tensor torch.Size([2, 4, 2, 3])


tensor([[[[235., 246., 242.],
          [266., 265., 247.]],

         [[223., 234., 230.],
          [254., 253., 235.]],

         [[215., 226., 222.],
          [246., 245., 227.]],

         [[221., 232., 228.],
          [252., 251., 233.]]],


        [[[270., 272., 287.],
          [258., 259., 272.]],

         [[258., 260., 275.],
          [246., 247., 260.]],

         [[250., 252., 267.],
          [238., 239., 252.]],

         [[256., 258., 273.],
          [244., 245., 258.]]]])

In [ ]:
img

In [ ]:
img[:,0:3,0:3]

In [ ]:
a = torch.randint(10, ( 3, 7, 10)).to(torch.float32)
input_width=a.shape[-1]
input_height=a.shape[-2]
kernel_size =5
stride = 1
padding=0


def image_shape(input_height, input_width, kernel_size, stride, padding):
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    return output_height, output_width

output_height,output_width=image_shape(input_height, input_width,kernel_size,stride, padding)

print(output_height,output_width)

In [ ]:
kernel = torch.tensor(
                  [[[[0., 1, 0],
                     [1,  2, 1],
                     [0,  1, 0]],

                    [[1, 2, 1],
                     [0, 3, 3],
                     [0, 1, 10]],

                    [[10, 11, 12],
                     [13, 14, 15],
                     [16, 17, 18]]]])

out_channels = kernel.shape[0]
in_channels  = kernel.shape[1]
kernel_size  = kernel.shape[2]
stride=2

print(f" \
out_channels {out_channels} \n \
in_channels  {in_channels}  \n \
kernel_size  {kernel_size}  \n \
"
)

In [ ]:
my_conv2d = Conv2dLoop(in_channels, out_channels, kernel_size, stride)
my_conv2d.set_kernel(kernel)

In [ ]:
input_tensor.shape[0]

In [ ]:
len(input_tensor.shape)

## Torch conv2d

In [ ]:
# in_channels = 3
# out_channels = 4
# kernel_size = 3
# stride = 1

in_channels = 3
out_channels = 4
kernel_size = 3
stride = 1

conv2d_torch = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

conv2d_torch(img).shape

In [ ]:
a = torch.randint(10, (2, 3, 7, 7)).to(torch.float32)
a.shape[0]
# for i in a:
#     print(i.shape)

In [ ]:
conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

In [ ]:
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)

In [ ]:
torch_conv2d = Conv2d(3,4,5,1)
torch_conv2d(a).shape
torch_conv2d(a).shape

In [ ]:
a[0].shape

In [ ]:
def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width

calc_out_shape(a.shape, 4, 5, 1, 0)

In [ ]:
## Test input 

In [ ]:
kernel = torch.tensor(
                  [[[[0., 1, 0],
                     [1,  2, 1],
                     [0,  1, 0]],

                    [[1, 2, 1],
                     [0, 3, 3],
                     [0, 1, 10]],

                    [[10, 11, 12],
                     [13, 14, 15],
                     [16, 17, 18]]]])

in_channels = kernel.shape[1]
in_channels

In [ ]:
batch_size=2
input_height=4
input_width=4
stride=2


print(batch_size * in_channels * \
                            input_height * input_width)

input_tensor = torch.arange(0, batch_size * in_channels *
                            input_height * input_width,
                            out=torch.FloatTensor()) \
    .reshape(batch_size, in_channels, input_height, input_width)
print(input_tensor.shape)
input_tensor.shape[-3]